In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
print(torch.cuda.is_available())  # Should print: True

True


In [ ]:
!pip install evaluate bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12


In [ ]:
!pip install evaluate

In [ ]:
import os
import csv
import evaluate

bertscore = evaluate.load("bertscore")

def read_text_file(path):
    with open(path, 'r', encoding='utf-8') as f:
        return f.read().strip()

def compute_bertscore_single_file(ref_path, hyp_path, lang="en"):
    gold = read_text_file(ref_path)
    hyp = read_text_file(hyp_path)

    result = bertscore.compute(predictions=[hyp], references=[gold], lang=lang)
    f1 = result["f1"][0] * 100  # Extract first item and convert to percentage
    return round(f1, 2)

def compute_bertscore_for_directory(gold_dir, hyp_dir, tag):
    scores = {}

    for hyp_file in os.listdir(hyp_dir):
        if hyp_file.endswith(".txt"):
            hyp_path = os.path.join(hyp_dir, hyp_file)
            meeting_id = hyp_file.split(".")[0]
            gold_file = f"{meeting_id}.gold_normalized.txt"
            gold_path = os.path.join(gold_dir, gold_file)

            if not os.path.isfile(gold_path):
                print(f"⚠️ No gold file for {hyp_file} found in {gold_dir}")
                continue

            score = compute_bertscore_single_file(gold_path, hyp_path)
            if meeting_id not in scores:
                scores[meeting_id] = {}
            scores[meeting_id][tag] = score
            print(f"✅ {meeting_id} — {tag} BERTScore (F1): {score:.2f}")

    return scores

def write_bertscore_to_csv(bertscore_scores, output_csv_path):
    os.makedirs(os.path.dirname(output_csv_path), exist_ok=True)

    with open(output_csv_path, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(["BERTScore F1"])
        writer.writerow(["File", "with_punct", "without_punct"])
        for file_name in sorted(bertscore_scores):
            s = bertscore_scores[file_name]
            row = [
                file_name,
                s.get("with_punct", ""),
                s.get("without_punct", "")
            ]
            writer.writerow(row)

if __name__ == "__main__":
    output_path = "/content/drive/MyDrive/Uni/FS25/evals/BERTScore_scores.csv"
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    # SCENARIO
    # order: tag, gold, hyp
    scenario_dirs = {
        "with_punct": (
            "/content/drive/MyDrive/Uni/FS25/gold/scenario/FULLY_NORMALIZED/normalized_p",
            "/content/drive/MyDrive/Uni/FS25/Whisper_outputs/scenario/FULLY_NORMALIZED/normalized_p"
        ),
        "without_punct": (
            "/content/drive/MyDrive/Uni/FS25/gold/scenario/FULLY_NORMALIZED",
            "/content/drive/MyDrive/Uni/FS25/Whisper_outputs/scenario/FULLY_NORMALIZED"
        )
    }

    # NATURAL
    natural_dirs = {
        "with_punct": (
            "/content/drive/MyDrive/Uni/FS25/gold/natural/FULLY_NORMALIZED/normalized_p",
            "/content/drive/MyDrive/Uni/FS25/Whisper_outputs/natural/FULLY_NORMALIZED/normalized_p"
        ),
        "without_punct": (
            "/content/drive/MyDrive/Uni/FS25/gold/natural/FULLY_NORMALIZED",
            "/content/drive/MyDrive/Uni/FS25/Whisper_outputs/natural/FULLY_NORMALIZED"
        )
    }

    bertscore_scores = {}

    for tag, (gold_dir, hyp_dir) in scenario_dirs.items():
        scores = compute_bertscore_for_directory(gold_dir, hyp_dir, tag)
        for k, v in scores.items():
            bertscore_scores.setdefault(k, {}).update(v)

    for tag, (gold_dir, hyp_dir) in natural_dirs.items():
        scores = compute_bertscore_for_directory(gold_dir, hyp_dir, tag)
        for k, v in scores.items():
            bertscore_scores.setdefault(k, {}).update(v)

    write_bertscore_to_csv(bertscore_scores, output_path)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ ES2016d — with_punct BERTScore (F1): 89.66
✅ ES2016b — with_punct BERTScore (F1): 91.10
✅ ES2016a — with_punct BERTScore (F1): 84.01
✅ ES2016c — with_punct BERTScore (F1): 93.98
✅ ES2016a — without_punct BERTScore (F1): 86.02
✅ ES2016b — without_punct BERTScore (F1): 93.74
✅ ES2016d — without_punct BERTScore (F1): 90.12
✅ ES2016c — without_punct BERTScore (F1): 95.83
✅ EN2009d — with_punct BERTScore (F1): 87.98
✅ EN2009c — with_punct BERTScore (F1): 84.66
✅ EN2009d — without_punct BERTScore (F1): 89.34
✅ EN2009c — without_punct BERTScore (F1): 85.98
